#### Agrupación / GroupBy()

La función groupby() de Pandas se utiliza para agrupar datos en un DataFrame según una o más columnas. Esto permite realizar operaciones agregadas, estadísticas y de análisis a nivel de grupo. Los principales usos de groupby() incluyen:

    1- Agrupación de Datos: Permite dividir los datos en grupos basados en los valores de una o más columnas.
    2- Aplicación de Funciones Agregadas: Después de agrupar los datos, se pueden aplicar funciones como sum(), mean(), count(), max(), min(), entre otras, para obtener resúmenes estadísticos de cada grupo.
    3- Análisis de Datos: Facilita el análisis comparativo entre diferentes grupos, permitiendo entender las tendencias y patrones en los datos.
    4- Manipulación de Datos: Permite transformar los datos dentro de cada grupo antes de combinarlos nuevamente en un DataFrame.

In [ ]:
import pandas as pd 
import numpy as np

df = pd.read_csv("../datasets/census.csv")
df = df[df["SUMLEV"] == 50]
df.head()

Vamos a realizar la misma operación utilizando dos sistemas diferentes, para ver cual es más óptimo y eficiente. Cuando lo ejecutemos podremos observar la diferencia abismal en tiempo de ejecución: Sin groupby(32s) con groupby (.2s)

In [ ]:
%%timeit -n 3

# ejemplo sin groupby()

for state in df["STNAME"].unique():
    avg = np.average(df.where(df["STNAME"]==state).dropna()["CENSUS2010POP"])
    print("Condados en el Estado " + state + " tienen una población media de " + str(avg))

In [ ]:
%%timeit -n 3

# ejemplo con groupby()
# groupby() devuelve una tupla donde el primer valor es valor de la clave por la que
# estamos agrupando, en este caso el nombre de un estado, y el segundo valor es el dataframe
# que se encontro para ese grupo

for group, frame in df.groupby("STNAME"):
    avg = np.average(frame["CENSUS2010POP"])
    print("Condados en el Estado " + group + " tienen una población media de " + str(avg))

In [ ]:
# tambien podemos pasarle una función a groupby() y usarla para segmentar los datos
# supongamos que tienes un gran trabajo por lotes con mucho procesamiento y quieres trabajar 
# solo con un tercio de los estados a la vez. Podríamos crear alguna función que devuelva un 
# número entre cero y dos basado en el primer carácter del nombre del estado. Luego, podemos 
# decirle a groupby que use esta función para dividir nuestro DataFrame. Es importante notar 
# que, para hacer esto, necesitas establecer el índice del DataFrame en la columna por la que 
# quieres agrupar primero.

# crearemos una nueva función llamada set_batch_number y si la primera letra del parámetro es 
# una "M" mayúscula, devolveremos un 0. Si es una "Q" mayúscula, devolveremos un 1 y, de lo 
# contrario, devolveremos un 2. Luego, pasaremos

df = df.set_index('STNAME')

def set_batch_number(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2

for group, frame in df.groupby(set_batch_number):
    print('Hay ' + str(len(frame)) + ' registros en el grupo ' + str(group) + ' para procesar.')

# en caso de no pasarle una columna como identificador, groupby utilizara el índice

In [ ]:
# vamos a cargar un conjunto de datos de AirBnB, el dataset incluye dos columnas cancellation_policy
# y review_scores_value

df=pd.read_csv("../datasets/listings.csv")
df.head()

In [ ]:
# para usar groupby en esas dos columnas tenemos diferentes formas de hacerlo.
# la primera sería utilizar un multi-index y llamar a groupby()

df = df.set_index(["cancellation_policy", "review_scores_value"])
df.head()

# cuando tenemos un indice multinivel tenemos que pasarle los niveles por los que 
# queremos agrupar

for group, frame in df.groupby(level=(0,1)):
    print(group)

In [ ]:
# si queremos agrupar por la política de cancelación y las puntuaciones de las reseñas, 
# pero separando todas las puntuaciones de 10 de las que están por debajo de 10, podemos 
# usar una función personalizada para gestionar estas agrupaciones.

def grouping_fun(item):
    if item[1] == 10:
        return (item[0],"10.0")
    else:
        return (item[0], "Not 10.0")

for group, frame in df.groupby(by=grouping_fun):
    print(group)

#### Agregación / agg()

El método .agg() en Pandas se utiliza para aplicar una o más funciones de agregación a los grupos de un DataFrame. Es particularmente útil después de usar groupby() para realizar operaciones agregadas en columnas específicas de cada grupo. Aquí están algunas de las principales ventajas y usos de .agg():

    1- Aplicar múltiples funciones: Puedes aplicar varias funciones de agregación a una o más columnas.
    2- Definir funciones personalizadas: Puedes usar funciones predefinidas como sum, mean, max, etc., o definir tus propias funciones personalizadas.
    3- Aplicar diferentes funciones a diferentes columnas: Puedes especificar diferentes funciones de agregación para diferentes columnas en el mismo comando.

In [ ]:
# para probar esto vamos a agrupar por politica de cancelación y encontrar el media de 
# las reviews con groupby

df = df.reset_index()

df.groupby("cancellation_policy").agg({"review_scores_value":np.average})

In [ ]:
# esto nos arrojará un error, dado que np.average no ignora los NaN, podemos solucionarlo
# utilizando una función para eso
# con np.nanmean excluimos los NaN
df.groupby("cancellation_policy").agg({"review_scores_value":np.nanmean})

In [ ]:
# podemos extender el diccionario si queremos agregar mñas de una función o columnas
df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean, np.nanstd),
                                       "reviews_per_month":np.nanmean})

# cuando utilizamos agg, le pasamos un diccionario: la clave sera la columna en la que
# queremos aplicar la función y el valor, la función a aplicar. Podemos pasar más de una 
# función por columna

#### Transformaciones / transform()
El método transform() en Pandas se utiliza para aplicar una función a un grupo o una columna y devolver un objeto que tiene el mismo tamaño que el grupo o columna original. Esto es especialmente útil cuando necesitas transformar datos a nivel de grupo pero quieres mantener la estructura original del DataFrame.

    1- Aplicar funciones de agregación y mantener el tamaño original: A diferencia de agg(), que devuelve un DataFrame reducido, transform() devuelve un objeto del mismo tamaño que los datos originales.
    2- Aplicar transformaciones específicas: Permite aplicar funciones personalizadas que realizan transformaciones a cada elemento del grupo.
    3- Operaciones dentro de grupos: Ideal para operaciones que necesitan comparar elementos dentro del mismo grupo, como estandarización o escalado.

In [ ]:
# primero vamos a definir las columnas en las que estamos interesados
columnas = ["cancellation_policy", "review_scores_value"] 

# aplicamos transform y guardamos los resultados en su propio dataframe
transform_df = df[columnas].groupby("cancellation_policy").transform(np.nanmean)
transform_df.head()

In [ ]:
# se renombra la columna review_scores_value a mean_scores_values en el DataFrame transform_df. 
# La opción axis="columns" especifica que se está renombrando una columna, y inplace=True indica 
# que el cambio se hará directamente en transform_df sin necesidad de crear una copia.

transform_df.rename({"review_scores_value":"mean_review_scores"}, axis="columns", inplace=True)

# se combinan los dataframes (haciendo un merge) df con transform_df utilizando los índices de ambos
# DataFrames como claves de combinación. Esto generalmente se hace cuando deseas añadir columnas de 
# transform_df a df basadas en sus índices.

df=df.merge(transform_df, left_index=True, right_index=True)

df['mean_review_scores'].head()

In [ ]:
# np.abs() es una función de NumPy que calcula el valor absoluto de la diferencia entre 
# df['review_scores_value'] y df['mean_review_scores'].

df['mean_diff']=np.abs(df['review_scores_value']-df['mean_review_scores'])
df['mean_review_scores'].head()


#### Filtrado / filter()
El método .filter() en Pandas se utiliza para filtrar filas o columnas de un DataFrame según ciertos criterios. Puede funcionar de dos maneras principales:

    1- Filtrado de columnas por nombre: Permite seleccionar un subconjunto de columnas en función de una lista de nombres.
    2- Filtrado condicional basado en etiquetas: Permite filtrar las filas del DataFrame usando etiquetas de fila.

In [ ]:
df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)

# permite filtrar el DataFrame df para incluir solo aquellas filas donde la media de las puntuaciones
# de revisión ('review_scores_value') dentro de cada grupo de política de cancelación ('cancellation_policy') 
# sea mayor que 9.2. Es útil para realizar análisis condicionales y seleccionar subconjuntos específicos de 
# datos basados en criterios de agregación dentro de grupos definidos.

#### Aplicación / apply()

El método .apply() en Pandas se utiliza principalmente para aplicar una función a elementos de una serie o DataFrame a lo largo de un eje específico. Puede funcionar de varias maneras:

    1- Aplicación de funciones a filas o columnas: Permite aplicar una función a cada fila o columna del DataFrame.
    2- Aplicación de funciones a elementos individuales: Permite aplicar una función a cada elemento individual de una serie.

In [ ]:
df=pd.read_csv("datasets/listings.csv")
df=df[['cancellation_policy','review_scores_value']]
df.head()

In [ ]:
def calc_mean_review_scores(group):
    avg=np.nanmean(group["review_scores_value"])
    group["review_scores_mean"]=np.abs(avg-group["review_scores_value"])
    return group

df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

# el DataFrame df por la columna 'cancellation_policy', luego para cada grupo calcula la media 
# de los valores en "review_scores_value" y crea una nueva columna "review_scores_mean" que 
# contiene la diferencia absoluta entre cada valor y la media del grupo respectivo. Esto es útil 
# para analizar cómo difieren las puntuaciones individuales de los valores medios dentro de cada 
# categoría de política de cancelación.